## Build the galaxy control tables, galaxy CDFs, and attach more information to the SNe sample

In [45]:
import numpy as np
import pandas as pd
import math
import os.path
sys.path.append(os.path.join(os.getenv('HOME'),'workspace','galbase'))
from gal_data import gal_data
import astropy.io.fits as pyfits
from astropy.wcs import WCS
from astropy import units as u
from astropy.utils.data import get_pkg_data_filename
from astropy.utils.console import ProgressBar
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from operator import itemgetter
from astropy.coordinates import SkyCoord
import astropy.units as u
from reproject import reproject_interp

In [61]:
galbase = pd.read_csv('samples/galbase_info.csv')
samp = pd.read_csv('samples/sne_sample.csv')

# fix
samp = samp.drop(columns=['Unnamed: 0'])
samp.to_csv('samples/sne_sample.csv', index=False)

In [48]:
from __future__ import (
    division, print_function, absolute_import, unicode_literals)


def deproject(center_coord=None, incl=0*u.deg, pa=0*u.deg,
              header=None, wcs=None, naxis=None, ra=None, dec=None,
              return_offset=False):

    """
    Calculate deprojected radii and projected angles in a disk.
    This function deals with projected images of astronomical objects
    with an intrinsic disk geometry. Given sky coordinates of the
    disk center, disk inclination and position angle, this function
    calculates deprojected radii and projected angles based on
    (1) a FITS header (`header`), or
    (2) a WCS object with specified axis sizes (`wcs` + `naxis`), or
    (3) RA and DEC coodinates (`ra` + `dec`).
    Both deprojected radii and projected angles are defined relative
    to the center in the inclined disk frame. For (1) and (2), the
    outputs are 2D images; for (3), the outputs are arrays with shapes
    matching the broadcasted shape of `ra` and `dec`.
    Parameters
    ----------
    center_coord : `~astropy.coordinates.SkyCoord` object or 2-tuple
        Sky coordinates of the disk center
    incl : `~astropy.units.Quantity` object or number, optional
        Inclination angle of the disk (0 degree means face-on)
        Default is 0 degree.
    pa : `~astropy.units.Quantity` object or number, optional
        Position angle of the disk (red/receding side, North->East)
        Default is 0 degree.
    header : `~astropy.io.fits.Header` object, optional
        FITS header specifying the WCS and size of the output 2D maps
    wcs : `~astropy.wcs.WCS` object, optional
        WCS of the output 2D maps
    naxis : array-like (with two elements), optional
        Size of the output 2D maps
    ra : array-like, optional
        RA coordinate of the sky locations of interest
    dec : array-like, optional
        DEC coordinate of the sky locations of interest
    return_offset : bool, optional
        Whether to return the angular offset coordinates together with
        deprojected radii and angles. Default is to not return.
    Returns
    -------
    deprojected coordinates : list of arrays
        If `return_offset` is set to True, the returned arrays include
        deprojected radii, projected angles, as well as angular offset
        coordinates along East-West and North-South direction;
        otherwise only the former two arrays will be returned.
    Notes
    -----
    This is the Python version of an IDL function `deproject` included
    in the `cpropstoo` package. See URL below:
    https://github.com/akleroy/cpropstoo/blob/master/cubes/deproject.pro
    """

    if isinstance(center_coord, SkyCoord):
        x0_deg = center_coord.ra.degree
        y0_deg = center_coord.dec.degree
    else:
        x0_deg, y0_deg = center_coord
        if hasattr(x0_deg, 'unit'):
            x0_deg = x0_deg.to(u.deg).value
            y0_deg = y0_deg.to(u.deg).value
    if hasattr(incl, 'unit'):
        incl_deg = incl.to(u.deg).value
    else:
        incl_deg = incl
    if hasattr(pa, 'unit'):
        pa_deg = pa.to(u.deg).value
    else:
        pa_deg = pa

    if header is not None:
        wcs_cel = WCS(header).celestial
        naxis1 = header['NAXIS1']
        naxis2 = header['NAXIS2']
        # create ra and dec grids
        ix = np.arange(naxis1)
        iy = np.arange(naxis2).reshape(-1, 1)
        ra_deg, dec_deg = wcs_cel.wcs_pix2world(ix, iy, 0)
    elif (wcs is not None) and (naxis is not None):
        wcs_cel = wcs.celestial
        naxis1, naxis2 = naxis
        # create ra and dec grids
        ix = np.arange(naxis1)
        iy = np.arange(naxis2).reshape(-1, 1)
        ra_deg, dec_deg = wcs_cel.wcs_pix2world(ix, iy, 0)
    else:
        ra_deg, dec_deg = np.broadcast_arrays(ra, dec)
        if hasattr(ra_deg, 'unit'):
            ra_deg = ra_deg.to(u.deg).value
            dec_deg = dec_deg.to(u.deg).value

    # recast the ra and dec arrays in term of the center coordinates
    # arrays are now in degrees from the center
    dx_deg = (ra_deg - x0_deg) * np.cos(np.deg2rad(y0_deg))
    dy_deg = dec_deg - y0_deg

    # rotation angle (rotate x-axis up to the major axis)
    rotangle = np.pi/2 - np.deg2rad(pa_deg)

    # create deprojected coordinate grids
    deprojdx_deg = (dx_deg * np.cos(rotangle) +
                    dy_deg * np.sin(rotangle))
    deprojdy_deg = (dy_deg * np.cos(rotangle) -
                    dx_deg * np.sin(rotangle))
    deprojdy_deg /= np.cos(np.deg2rad(incl_deg))

    # make map of deprojected distance from the center
    radius_deg = np.sqrt(deprojdx_deg**2 + deprojdy_deg**2)

    # make map of angle w.r.t. position angle
    projang_deg = np.rad2deg(np.arctan2(deprojdy_deg, deprojdx_deg))

    if return_offset:
        return radius_deg, projang_deg, dx_deg, dy_deg
    else:
        return radius_deg, projang_deg

### Build the galaxy control tables

Function that will construct the actual table.

In [46]:
def control_table(pix_values, rms_values, pgcs, ra, dec, radii, ra_w4, dec_w4, radii_w4):
    
    var_pix = pd.DataFrame()
    var_pix['W1'] = pix_values[0]
    var_pix['W2'] = pix_values[1]
    var_pix['W3'] = pix_values[2]
    var_pix['NUV'] = pix_values[4]
    var_pix['FUV'] = pix_values[5]
    
    var_rms = pd.DataFrame()
    var_rms['RMS_W1'] = rms_values[0]
    var_rms['RMS_W2'] = rms_values[1]
    var_rms['RMS_W3'] = rms_values[2]
    var_rms['RMS_NUV'] = rms_values[4]
    var_rms['RMS_FUV'] = rms_values[5]
    
    # pixel information
    pgcs_df = pd.DataFrame(pgcs, columns=['PGC'])
    ras_df = pd.DataFrame(ra, columns=['RA']) # ra and dec will act as pixel identifier
    decs_df = pd.DataFrame(dec, columns=['DEC'])
    radii_df = pd.DataFrame(radii, columns=['RADIUS'])
    
    ras_w4_df   = pd.DataFrame(ra_w4, columns=['RA_W4'])
    decs_w4_df  = pd.DataFrame(dec_w4, columns=['DEC_W4'])
    radii_w4_df = pd.DataFrame(radii_w4, columns=['RADIUS_W4'])
    w4_df       = pd.DataFrame(pix_values[3], columns=['W4'])
    rms_w4_df   = pd.DataFrame(rms_values[3], columns=['RMS_W4'])
    
    fuv_on_w4_df = pd.DataFrame(pix_values[6], columns=['FUV_ON_W4'])
    rms_fuv_on_w4_df = pd.DataFrame(rms_values[6], columns=['RMS_FUV_ON_W4'])
    
    total = pd.concat([ras_df, decs_df, ras_w4_df, decs_w4_df, pgcs_df, radii_df, radii_w4_df, 
                       var_pix, w4_df, fuv_on_w4_df, var_rms, rms_w4_df,rms_fuv_on_w4_df], axis=1)
    total = total[['RA','DEC','RA_W4','DEC_W4','PGC','RADIUS','RADIUS_W4','W1','W2', 'W3', 'NUV','FUV',
                   'W4','FUV_ON_W4','RMS_W1','RMS_W2','RMS_W3','RMS_NUV','RMS_FUV','RMS_W4','RMS_FUV_ON_W4']]
    return(total)

Function to sort flux by radius

In [47]:
def sorting_flux_by_radius(gal_flux, gal_rad):
    """ Input: galaxy flux, galaxy radius at pixel
        Output: radius value, CDF value at pixel
    """
    flux_rad_zipped = zip(gal_flux, gal_rad)
    flux, rad = zip(*sorted(flux_rad_zipped, key=itemgetter(1)))
    return(rad, np.cumsum(flux)/np.sum(flux))

Function that will convert the galaxy's pixel coordinates to RAs and DECs in degrees

In [30]:
def gal_pixel_coords(wcs):
    naxis = wcs._naxis
    x = np.arange(naxis[0])
    y = np.arange(naxis[1])
    coords_arr = np.column_stack((x, y))
    ras_decs = wcs.wcs_pix2world(coords_arr, 0)
    gal_ras = ras_decs[:,0]
    gal_decs = ras_decs[:,1]
    return(gal_ras, gal_decs)

Function that will loop over each galaxy in our sample and grab all the pixel values and radii and will store them in the galaxy control tables

In [31]:
def radius_map(hdulist, res, pgc, r25_deg, w4):
    
    # radius map for all but w4
    w = WCS(hdulist[0].header)
    mapp = hdulist[0].data.copy()
    dimx, dimy = np.shape(mapp)
    mapp = mapp.flatten()
    
    if w4 is not True:
        hdulist1   = pyfits.open('rgrid/%s/%s_%s_rgrid.fits' % (res,pgc,res))
    else:
        hdulist1   = pyfits.open('rgrid/%s/%s_%s_w4_rgrid.fits' % (res,pgc,res))
        
    rad_map    = hdulist1[0].data.copy() # copy data out into variable
    w          = WCS(hdulist1[0].header)
    rad_map    = rad_map.flatten()
    loc_inside = np.where(rad_map <= 2*r25_deg)[0]
    radii      = rad_map[loc_inside].byteswap().newbyteorder() # radii array; big endian to little endian (fixes error)
    hdulist1.close()

     # RA, DEC MAP
    xx, yy = np.meshgrid(range(dimx), range(dimy))
    xx = xx.flatten()
    yy = yy.flatten()

    xx_inside = xx[loc_inside]
    yy_inside = yy[loc_inside]
    ra, dec   = w.wcs_pix2world(xx_inside, yy_inside, 0) # ra and dec arrays
    return(ra, dec, radii, loc_inside)

Build the galaxy control tables.

In [32]:
# collect all pixel values of galaxy
def gal_tables(samp, res):
    bar = ProgressBar(len(samp), ipython_widget=True)

    gal_in_process = []
    for index, row in samp.iterrows():
        pgc     = row['PGC']
        r25_deg = row['R25']
        ra_gal  = row['RA_GAL']
        dec_gal = row['DEC_GAL']
        incl    = row['INCL']
        pa      = row['PA']

        # if this galaxy was already accounted for    
        if pgc in gal_in_process:
            bar.update()
            continue
        gal_in_process.append(pgc)

        try:
            hdulist = pyfits.open('/data/kant/0/leroy.42/allsky/convolved/%s/%s_w1_%s.fits' % (res, pgc, res))
            ra, dec, radii, loc_inside = radius_map(hdulist, res, pgc, r25_deg, False)
            hdulist.close()
        except:
            bar.update()
            continue
            
        try:
            hdulist_w4 = pyfits.open('/data/kant/0/leroy.42/allsky/convolved/%s/%s_w4_%s.fits' % (res, pgc, res))
            ra_w4, dec_w4, radii_w4, loc_inside_w4 = radius_map(hdulist_w4, res, pgc, r25_deg, True)
            hdulist_w4.close()
        except:
            ra_w4  = np.array([np.nan] * len(ra))
            dec_w4 = np.array([np.nan] * len(ra))
            radii_w4 = np.array([np.nan] * len(ra))
            loc_inside_w4 = np.array([np.nan] * len(ra))
        
        
        pgcs = [pgc] * len(ra) # pgc array; want length of ra so all values are filled
        
        # create a matrix that contains indiviudal arrays for each band
        pix_values   = [[],[],[],[],[],[],[]]
        rms_values   = [[],[],[],[],[],[],[]]

        # now loop over all bands
        bands = ['w1','w2','w3','w4','nuv','fuv', 'fuv_on_w4']
        for bb in range(len(bands)):

            # if there are file not found errors, then try another file. If another error, then write 'nan'
            try:
                if bands[bb] == 'fuv_on_w4':
                    hdulist1 = pyfits.open('rgrid/fuv_on_w4_%s/%s_fuv_on_w4_%s.fits' % (res, pgc, res))
                
                else:
                    hdulist1 = pyfits.open('/data/kant/0/leroy.42/allsky/convolved/%s/%s_%s_%s.fits' % (res, pgc, bands[bb], res))
               
                band_map = hdulist1[0].data
                rms = np.array([hdulist1[0].header['RMS']] * len(ra)) # multiply by length of ra so this accounts for all pixels
                band_map = band_map.flatten()
                
                # get pixel values
                if bands[bb] == 'w4':
                    values = band_map[loc_inside_w4]
                elif bands[bb] == 'fuv_on_w4':
                    values = band_map[loc_inside_w4]
                else:
                    values = band_map[loc_inside]
                    
                values = values.byteswap().newbyteorder() # big endian to little endian (fixes error)
                hdulist1.close()
            except:
                values = np.array([np.nan] * len(ra))
                rms    = np.array([np.nan] * len(ra))

            pix_values[bb]   = values
            rms_values[bb]   = rms

        total = control_table(pix_values, rms_values, pgcs, ra, dec, radii, ra_w4, dec_w4, radii_w4)
        total.to_csv('galaxy_control_tables/%s/%s_%s.csv' % (res, pgc, res), index=False)
        bar.update()
    return

In [33]:
gal_tables(samp, '500pc')
gal_tables(samp, '1kpc')
gal_tables(samp, '2kpc')

FloatProgress(value=0.0)

FloatProgress(value=0.0)

FloatProgress(value=0.0)

#### Now repeat for the SFR tracer

Calculate the SFR and add a column to the galaxy control tables.

In [34]:
# W4; C = 10**-42.73
def w4_SFR_tracer(pixel_flux_w4):
    """ Input: flux measured at pixel (W4)
        Output: surfance density of SFR 
        in M_sol yr-1 kpc-2
    """
    SFR_w4 = (3.24e-03)*((10**(-42.73)) / (10**(-42.7)))*(pixel_flux_w4)
    return(SFR_w4)

# FUV; C = 10**-43.42
def fuv_SFR_tracer(pixel_flux_fuv):
    """ Input: flux measured at pixel (FUV)
        Output: surfance density of SFR 
        in M_sol yr-1 kpc-2
    """
    SFR_fuv = (1.04e-01)*((10**(-43.42)) / (10**(-43.35)))*(pixel_flux_fuv)
    return(SFR_fuv)

Add the SFR tracer info to the galaxy control tables.

In [35]:
def gal_tables_sfr(res):

    bar = ProgressBar(len(os.listdir('galaxy_control_tables/%s/' % (res))), ipython_widget=True)

    # loop over every filename in the directory
    for filename in os.listdir('galaxy_control_tables/%s/' % (res)):

        # open the files
        control_table = pd.read_csv('galaxy_control_tables/%s/%s' % (res, filename))   
        pgc = filename.split('_%s.csv' % (res))[0]

        SFR_vals = []
        for index, row in control_table.iterrows():

            # grab each flux value
            pixel_flux_w4  = row['W4']
            pixel_flux_fuv = row['FUV_ON_W4']

            # convert w4 and fuv to SFR tracer
            SFR_w4  = w4_SFR_tracer(pixel_flux_w4)
            SFR_fuv = fuv_SFR_tracer(pixel_flux_fuv)
            SFR_tracer = SFR_w4 + SFR_fuv

            SFR_vals += [SFR_tracer]

        # add the columns
        control_table['SFR'] = SFR_vals

        # rearrange columns
        control_table = control_table[['RA','DEC', 'RA_W4', 'DEC_W4','PGC','RADIUS','RADIUS_W4','W1','W2','W3','W4',
                                       'NUV','FUV','FUV_ON_W4','SFR','RMS_W1','RMS_W2','RMS_W3','RMS_W4','RMS_NUV',
                                       'RMS_FUV','RMS_FUV_ON_W4']]
        # overwrite old control tables
        control_table.to_csv('galaxy_control_tables/%s/%s_%s.csv' % (res, pgc, res), index=False)

        bar.update()
        
    return

In [36]:
gal_tables_sfr('500pc')
gal_tables_sfr('1kpc')
gal_tables_sfr('2kpc')

FloatProgress(value=0.0)

FloatProgress(value=0.0)

FloatProgress(value=0.0)

### Build the galaxy CDFs

Function that will grab each galaxy's control table and will build the host galaxy CDF

In [37]:
def build_galaxy_cdf(band, pgc, res):
    """ Input: bands, pgc
        Output: rad (radius), cdf (cdf value)
        This function makes cuts based on
        nan values. It then uses the function 
        sorting_flux_by_radius() to compute and 
        return the radius and galaxy CDF value at the location.
    """
    
    # grab the control table
    gal_info = pd.read_csv('galaxy_control_tables/%s/%s_%s.csv' % (res, pgc, res))
    gal_info_band = np.array(gal_info[band])

    if band == 'W4':
        gal_rad = gal_info['RADIUS_W4']
    elif band == 'SFR':
        gal_rad = gal_info['RADIUS_W4']
    else:
        gal_rad = gal_info['RADIUS']
        
        
    # get rid of nan
    gal_rad = gal_rad[np.isfinite(gal_info_band)]
    gal_info_band = gal_info_band[np.isfinite(gal_info_band)]

    # set negative values to 0
    loc_negative = np.where(gal_info_band < 0)[0]
    gal_info_band[loc_negative] = 0.
    
    # call function to build CDF
    rad, cdf = sorting_flux_by_radius(gal_info_band, gal_rad)
    
    return(rad, cdf)

Function that will output the galaxy CDF to a file

In [38]:
def gal_cdf_to_file(rad, cdf, pgc, band, res):
    """ Input: radius, cdf values, galaxy pgc, and bands
        Output: individual files of the radii and cdf
        values per galaxy per band
    """
    
    gal_cdf_dict = {
        'RADIUS': rad,
        'CDF'   : cdf
    }
 
    gal_cdf = pd.DataFrame(gal_cdf_dict)
    gal_cdf.to_csv('galaxy_control_tables/gal_cdfs_%s/%s_%s_%s.csv' % (res, pgc, band, res), index=False)

    return

Function that puts everything together: loop through the sample, build the galaxy CDF, and output to a file

In [39]:
def output_cdfs(samp, res, sfr):

    if sfr is not True:
        bands = ['W1', 'W2', 'W3', 'W4', 'NUV', 'FUV', 'SFR']
    else:
        bands = ['SFR']

    bar = ProgressBar(len(samp), ipython_widget=True)
    
    for index, row in samp.iterrows():

        # read in host galaxy control table
        pgc = row['PGC']

        # BUILD GALAXY CDF: normalize total flux summed over all pixels
        for i in range(len(bands)):

            try:
                # call function to make cuts and build galaxy CDF
                rad, cdf = build_galaxy_cdf(bands[i], pgc, res)

                # make these lists or else pandas gets mad
                rad = [rad for rad in rad]
                cdf = [cdf for cdf in cdf]
                
            except:
                continue

            # call function to write CDF and radius values to a file
            gal_cdf_to_file(rad, cdf, pgc, bands[i], res)
                
        bar.update()

In [41]:
# output_cdfs(samp, '500pc', False)
# output_cdfs(samp, '1kpc', False)
# output_cdfs(samp, '2kpc', False)

output_cdfs(samp, '500pc', True)
output_cdfs(samp, '1kpc', True)
output_cdfs(samp, '2kpc', True)

FloatProgress(value=0.0)

FloatProgress(value=0.0)

FloatProgress(value=0.0)

### Prep the sample to build the SNe CDFs
- Get the distance to each SN from the galactic center
- Measure flux values at each SN site (?????)

In [42]:
def add_radii_column_conv(samp, res):

    radii  = []
    names  = []

    bar = ProgressBar(len(samp), ipython_widget=True)
    for index, row in samp.iterrows():
        ra   = row['RA']
        dec  = row['DEC']
        ra_gal  = row['RA_GAL']
        dec_gal = row['DEC_GAL']
        pa = row['PA']
        incl = row['INCL']
        pgc  = row['PGC']
        name = row['NAME']
        
        rgal_list, phigal_list = deproject(center_coord=(ra_gal, dec_gal), incl=incl, pa=pa, ra=ra, dec=dec)
        radii += [rgal_list]
        names += [name]
        bar.update()
    return(radii, names)

Get the radii for each resolution. Add this radii as a column for each resolution's sample

In [58]:
samp_1kpc = pd.read_csv('samples/sne_sample.csv')
radii, names = add_radii_column_conv(samp, '1kpc')
samp_1kpc['SN_RAD'] = radii
samp_1kpc = samp_1kpc.drop(columns=['Unnamed: 0'])
samp_1kpc.to_csv('sample_processing/1kpc.csv', index=False)

samp_2kpc = pd.read_csv('samples/sne_sample.csv')
radii, names = add_radii_column_conv(samp, '2kpc')
samp_2kpc['SN_RAD'] = radii
samp_2kpc = samp_2kpc.drop(columns=['Unnamed: 0'])
samp_2kpc.to_csv('sample_processing/2kpc.csv', index=False)

samp_500pc = pd.read_csv('samples/sne_sample.csv')
radii, names = add_radii_column_conv(samp, '500pc')
samp_500pc['SN_RAD'] = radii
samp_500pc = samp_500pc.drop(columns=['Unnamed: 0'])
samp_500pc.to_csv('sample_processing/500pc.csv', index=False)

FloatProgress(value=0.0)

FloatProgress(value=0.0)

FloatProgress(value=0.0)